In [1]:
import os
import sys
import jsonlines
from loguru import logger
from pathlib import Path
home_path = Path().absolute().parents[1]
logger.info(f"home path: {home_path.as_posix()}")

logger.info('设置推理卡片为1,2')
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'
sys.path.append(home_path.as_posix())

from ultrarag.modules.llm import OpenaiLLM
from ultrarag.modules.database import QdrantIndex
from ultrarag.modules.embedding import MiniCPMEmbServer


prompt = "请结合召回结果回答用户问题：\n召回结果：\n{recalls}\n用户问题：\n{query}\n回答："

corpus_path = home_path / 'examples' / 'workflow' / 'corpus.jsonl'
data = list(jsonlines.open(corpus_path.as_posix(), 'r'))


embedding_model_path = "/mnt1/guodewen/models/MiniCPM-Embedding-Light"
qdrant_collection_path = (home_path / 'resource' / 'qdrant').as_posix()

2025-04-16 17:06:02.777 | INFO     | __main__:<module>:7 - home path: /mnt1/guodewen/research/UltraRAG
2025-04-16 17:06:02.777 | INFO     | __main__:<module>:9 - 设置推理卡片为1,2


INFO 04-16 17:06:06 __init__.py:183] Automatically detected platform cuda.


/mnt1/guodewen/research/UltraRAG/ultrarag/modules/database/__init__.py:7: UserWarning: failed to load MilvusIndex beacause not available qdrant_client, ignored it if you do not need it
  warnings.warn("failed to load MilvusIndex beacause not available qdrant_client, ignored it if you do not need it")


In [2]:
llm = OpenaiLLM(
    model="QwQ-32B",
    base_url="http://localhost:8000/v1",
    api_key="EMPTY",
)

embed = MiniCPMEmbServer(url_or_path=embedding_model_path)

index = QdrantIndex(url=qdrant_collection_path, encoder=embed)

2025-04-16 17:06:12.358 | INFO     | ultrarag.modules.llm.openai_like:__init__:19 - api_key: EMPTY, base_url: http://localhost:8000/v1, kargs: {'model': 'QwQ-32B'}
2025-04-16 17:06:14.290 | INFO     | ultrarag.modules.embedding.minicpm_embedding:load_model:145 - The parameters of colbert_linear and sparse linear is new initialize. Make sure the model is loaded for training, not inferencing


----------using 2*GPUs----------


In [ ]:
if "demo" not in index.get_collections():
    await index.create("demo")
    
await index.insert(collection="demo", payloads=data, func=lambda x: x["title"] + x["content"])

In [ ]:
query="宫保鸡丁的做法"

results = await index.search(collection="demo", query=query, top_k=3)
logger.info(f"召回结果：\n{results}")

message = [
    dict(role="user", content=prompt.format(query=query, recalls=results)),
]
response = await llm.arun(messages=message, stream=False)
logger.info(f"推理结果：\n{response}")

2025-04-16 17:06:53.297 | INFO     | __main__:<module>:8 - 召回结果：
[BaseNode(score=0.6624137391062277, content='9. **宫保鸡丁**\\n\\n**食材准备**：鸡胸肉250g、花生米一小碗、干辣椒10个、花椒适量、葱姜蒜各适量、生抽、老抽、醋、糖、淀粉。\\n\\n**做法详解**：  \\n鸡胸肉洗净切成1厘米见方的小丁，用料酒、生抽、淀粉腌制15分钟，帮助鸡肉保持嫩滑口感。干辣椒剪成小段，花生米提前用小火炒熟备用。调料汁：生抽1勺、老抽半勺、糖1勺、醋1.5勺、水适量、淀粉1勺，搅拌均匀成宫保汁。\\n\\n锅中加油烧热，下鸡丁快速翻炒至变色后盛出。锅中留底油，放入干辣椒和花椒，用小火炒出香味（不要糊），然后加入葱姜蒜炒香，接着倒入鸡丁快速翻炒。最后淋入宫保汁，翻炒至浓稠，再撒入炒熟的花生米，炒匀即可。\\n\\n**小贴士**：干辣椒先炒香但不能炒糊；宫保汁一次倒入，快速翻炒收汁；花生最后放入保持脆感。', payload={'title': '宫保鸡丁', 'content': '9. **宫保鸡丁**\\n\\n**食材准备**：鸡胸肉250g、花生米一小碗、干辣椒10个、花椒适量、葱姜蒜各适量、生抽、老抽、醋、糖、淀粉。\\n\\n**做法详解**：  \\n鸡胸肉洗净切成1厘米见方的小丁，用料酒、生抽、淀粉腌制15分钟，帮助鸡肉保持嫩滑口感。干辣椒剪成小段，花生米提前用小火炒熟备用。调料汁：生抽1勺、老抽半勺、糖1勺、醋1.5勺、水适量、淀粉1勺，搅拌均匀成宫保汁。\\n\\n锅中加油烧热，下鸡丁快速翻炒至变色后盛出。锅中留底油，放入干辣椒和花椒，用小火炒出香味（不要糊），然后加入葱姜蒜炒香，接着倒入鸡丁快速翻炒。最后淋入宫保汁，翻炒至浓稠，再撒入炒熟的花生米，炒匀即可。\\n\\n**小贴士**：干辣椒先炒香但不能炒糊；宫保汁一次倒入，快速翻炒收汁；花生最后放入保持脆感。'}), BaseNode(score=0.3389509456255837, content='2. **青椒土豆丝**\\n\\n**食材准备**：土豆2个、青椒1个、大蒜2瓣、白醋1勺、盐适量、食用油适量。\\n\\n**做法详解**：  \\n将土豆去皮，先切成薄片

In [5]:
index.delete_collection("demo")